# Informed search - the A* algorithm

Depth-first search and breadth-first search considered in the previous lesson are completely *blind* algorithms: they're only concerned whether the currently considered state is a goal state or not. They're unable to distinguish whether a state is easy or hard to reach, or whether it is near or far of the goal. This makes them very inefficient search algorithms. To allievate the issue, we introduce informed search algorithms. The information is given to an algorithm in two ways:

1. By using an *action cost* function $c(s,a)$, which, given a state $s$ and an action $a$ available in this state, returns its cost as a non-negative number.
2. By using a *heuristic* $h(s)$, which, given a state, estimates the lowest cost to reach a goal state from the given state.

Given a sequence of actions $a_1, \ldots, a_n$ and an initial state $s_1$, we can express the *total cost* of reaching the state $s_{n+1}$ by executing the sequence as:
$$ c(s_1, a_1, \ldots, a_{n-1}) = \sum_{i=1}^n c(s_i, a_i) $$
and the *expected cost* of the solution as the sum of the total cost and the estimate cost of reaching the goal from the state $s_{n+1}$
$$ f(s_1, a_1, \ldots, a_n) = c(s_1, a_1, \ldots, a_n) + h(s_{n+1}) $$

The heuristic function is a bit tricky, because we want it to have two properties:
* *Admissibility*: It must never *overestimate* the true cost of reaching the goal. 
* *Consistency*: Let $s$ be a state such that $a$ is an available action in this state and $s'$ is the state reached by executing this action. The heuristic should fulfil triangle inequality, that is, the estimated cost to reach the goal from $s$ should be no greater than the cost of executing the action $a$ + the estimated cost of reaching the goal from the new state.
$$ h(s) \leq c(s, a) + h(s') $$

One can prove that admissibility follows from consistency, but consistency is important only if there are multiple paths to reach the same state (i.e., we are searching in a graph, not in a tree). Otherwise, admissability is sufficient.

Lets extend the class `Problem` from the previous lesson with two new functions `action_cost` and `heuristic`, which correspond to the functions $c(s,a)$ and $h(s)$ described above.

In [1]:
class Problem:
    @property
    def initial_state(self):
        ...
        
    def available_actions(self, state):
        ...        
        
    def do_action(self, state, action):
        ...
        return new_state
    
    def is_goal(self, state) -> bool:
        ...
        
    def action_cost(self, state, action) -> float:
        ...
        
    def heuristic(self, state) -> float:
        ...

To make a concrete example, lets revisit the vacuum world. 

![](aima-images/fig2_2.png)

Below, we assume a very simple model:
* Any action costs 1. This corresponds to searching for the shortest plan.
* The heuristic estimation is the number of fields which are still dirty. 


Lets consider the properties of the heuristic:
* Is is admissible? The heuristic value is equal to the number of 'Suck' actions that are yet to be executed and ignores the spatial aspect (i.e., moving between the rooms), thus never overestimating.
* Is it consistent? As a consequence of a single action the heuristic value can decrease by at most 1 (if the action happens to be 'Suck' and the room is dirty). The cost of any action is 1, so rewriting the triangle inequality we arrive at:
$$ h(s) \leq c(s, a) + h(s') = \begin{cases} 1 + (h(s)-1) & a=\text{'Suck' and the room was dirty} \\ 1 + h(s) & \text{otherwise} \end{cases} $$
* Is it the best we could have? By no means! We could include the spatial aspect.

In [2]:
class VacuumProblem(Problem):
    @property
    def initial_state(self):
        return (0, (True, True))
    
    def available_actions(self, state):
        return ["Left", "Suck", "Right"]
        
    def do_action(self, state, action):
        robot, dirty = state
        if action == "Left":
            return (max(robot-1, 0), dirty)
        elif action == "Suck":
            new_dirty = list(dirty)
            new_dirty[robot] = False
            return (robot, tuple(new_dirty))
        elif action == "Right":
            return (min(robot+1, len(dirty)-1), dirty)        
        raise Exception('Invalid action')
    
    def is_goal(self, state) -> bool:
        return not any(state[1])
    
    def action_cost(self, state, action):
        return 1
    
    def heuristic(self, state):
        return sum(state[1])

## Task 1: Implement the A* algorithm

To implement the A* algorithm you must have a priority queue. Luckily, Python comes with one, so you don't need to implement it by yourself. Then, the algorithm is very simple: 
1. Start with a queue containing a single item - the initial state
2. Repeat until the queue is not empty:
  1. Pick an item with the lowest expected cost
  2. If this is the goal, return the sequence of actions necessary to reach this state
  3. Otherwise, for each available action, create a new entry in the queue corresponding to the state reached after executing the action.
  
Guard the algorithm against infinite loops: if you already visited a state, you don't need to visit it again (if your heuristic is consistent).

In the cell below implement the algorithm in a similar manner as the BFS and DFS in the previous lesson: the sole argument is an object of the class Problem and the function should return a list of actions to achieve a goal state from the initial state.
If it is impossible to reach the goal, return `None`.
Count the number of states visited during the search and print in out before returning from the function, it will be useful later on to compare different heuristics.

In [3]:
from queue import PriorityQueue

In [4]:
def astar(problem: Problem):
    queue = PriorityQueue()
    in_state = problem.initial_state
    queue.put((0, in_state))
    visited = [in_state]
    actions = {in_state:[]}
    while not queue.empty():
        low = queue.get()
        cur_state = low[1]
        if problem.is_goal(cur_state): 
            print("The number of states visited:", len(visited))
            return actions[cur_state]
        for act in problem.available_actions(cur_state):
            new_entry = problem.do_action(cur_state, act)
            if new_entry not in visited:
                exp_cost = problem.action_cost(cur_state, act) + problem.heuristic(cur_state)
                queue.put((exp_cost, new_entry))
                actions[new_entry] = actions[cur_state][:]
                actions[new_entry].append(act)
                visited.append(new_entry)
    return None

Now lets test your code in the vacuum world!

In [5]:
astar(VacuumProblem())

The number of states visited: 5


['Suck', 'Right', 'Suck']

## Task 2: Variants of the vacuum world

Now lets consider a different take on the vacuum world in which the heuristic is not admissible and increases as the number of dirty fields decreases.

In [6]:
class VacuumProblem1(VacuumProblem):
    def action_cost(self, state, action):
        return 1
    
    def heuristic(self, state):
        return len(state[1]) - sum(state[1])
    
astar(VacuumProblem1())

The number of states visited: 7


['Right', 'Suck', 'Left', 'Suck']

And another in which heuristic grossly overestimates the cost.

In [7]:
class VacuumProblem2(VacuumProblem):
    def action_cost(self, state, action):
        return 1
    
    def heuristic(self, state):
        return 10 * sum(state[1])
    
astar(VacuumProblem2())

The number of states visited: 5


['Suck', 'Right', 'Suck']

**Which of the three heuristic functions (`VacuumProblem`, `VacuumProblem1`, `VacuumProblem2`) is the best? Is it the expected answer given the properties of the heuristics? If not, explain why an unorthodox approach works better.**

*Considering the obtained results, it can be concluded that the heuristic functions `VacuumProblem` and `VacuumProblem2` give the same results that are much better than one given by VacuumProblem1, which makes them **the best** in this case.* 

*As for properties of the heuristics, `VacuumProblem` works the best since `VacuumProblem2` largly overestimates its true cost from time to time which does not satisfy the heuristic property of admissibility (which is why the unorthodox approach works better).*

## Task 3: 8-puzzle problem

Recall the 8-puzzle problem from the previous lesson. Reuse your code and implement an extended version assuming that each action costs 1. Propose 3 (at least) admissible heuristics. This time don't change the initial state, your solution should be capable enough to solve this.

![](aima-images/fig3_4.png)

In [8]:
class PuzzleProblem(Problem):
    @property
    def initial_state(self):
        return (7, 2, 4, 5, 0, 6, 8, 3, 1)
        
    def available_actions(self, state):
        return ['Up', 'Down', 'Left', 'Right']       
        
    def do_action(self, state, action):
        new_state = list(state)
        cell = new_state.index(0)
        if action == 'Up' and cell in [3, 4, 5, 6, 7, 8]:
            new_state[cell - 3] = state[cell]
            new_state[cell] = state[cell - 3]
        elif action == 'Down' and cell in [0, 1, 2, 3, 4, 5]:
            new_state[cell] = state[cell + 3]
            new_state[cell + 3] = state[cell]
        elif action == 'Left' and cell in [1, 2, 4, 5, 7, 8]:
            new_state[cell] = state[cell - 1]
            new_state[cell - 1] = state[cell]            
        elif action == 'Right' and cell in [0, 1, 3, 4, 6, 7]:
            new_state[cell] = state[cell + 1]
            new_state[cell + 1] = state[cell]            
        return tuple(new_state)
        raise Exception('Invalid action')
    
    def is_goal(self, state) -> bool:
        if state == (0, 1, 2, 3, 4, 5, 6, 7, 8):
            return True
        else:
            return False
        
    def action_cost(self, state, action) -> float:
        return 1
        
    def heuristic(self, state) -> float:
        count = 0
        goal = [0, 1, 2, 3, 4, 5, 6, 7, 8]
        for i in range(len(state)):
            if state[i] != goal[i] and state[i] != 0:
                count += 1
        return count

**Prove that this heuristic is admissible.**

*Since the number of misplaced cells was used as a heuristic value, it will never overestimate the true cost (i.e., each cell will be definitely moved at least once to get to its goal cell).*

In [9]:
class PuzzleProblem1(PuzzleProblem):
    def heuristic(self, state) -> float:     
        goal = [0, 1, 2, 3, 4, 5, 6, 7, 8]
        return sum(abs(i - j) for i, j in zip(state, goal))

**Prove that this heuristic is admissible.**

*Since sum of Manhattan distances was used as a heuristic value, it always takes the shortest path to a cell's goal cell (i.e., it is possible for a cell only to move closer to its goal by one step in each move).*

In [12]:
class PuzzleProblem2(PuzzleProblem):
    def heuristic(self, state) -> float:
        goal = [0, 1, 2, 3, 4, 5, 6, 7, 8]
        axis = [[x, y] for x in range(1, 4) for y in range(1, 4)]
        coord_ini = sorted({state[i]:axis[i] for i in range(len(state))}.items())
        coord_goal = sorted({goal[i]:axis[i] for i in range(len(goal))}.items())
        dists = []
        for i in range(len(state)):
            dist = (((coord_goal[i][1][0] - coord_ini[i][1][0]) ** 2) + ((coord_goal[i][1][1] - coord_ini[i][1][1]) ** 2))*(1/2)
            dists.append(dist)
        return sum(dists)

**Prove that this heuristic is admissible.**

*Since sum of Euclidian distances was used as a heuristic value, it always takes the shortest path to a cell's goal cell (i.e., it is possible for a cell only to move closer to its goal by one step in each move (similar to the sum of Manhattan distances). Apart from that, the Eucledian distance is never greater than the amount of steps to move a cell to its goal cell).*

Run your heuristics on the given problem.

In [13]:
plan0 = astar(PuzzleProblem())
plan1 = astar(PuzzleProblem1())
plan2 = astar(PuzzleProblem2())

print("Is plan0==plan1?", plan0 == plan1)
print("Is plan0==plan2?", plan0 == plan2)
print("Is plan1==plan2?", plan1 == plan2)

The number of states visited: 1430
The number of states visited: 519
The number of states visited: 215
Is plan0==plan1? False
Is plan0==plan2? False
Is plan1==plan2? False


**Which of the heuristics is the best for this task? Why is that?**

*According to the results, it can be seen that `plan2` performs in less amount of steps than the other heuristic approaches, which makes it **the best** for this task (since all of them are considered admissible).*

------------
The pictures and the description of 8-puzzle are from "Artificial Intelligence: A Modern Approach" 3rd ed.